In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Exercise 1:

### Google Trends presentation

Imagine you are asked to put together a presentation of how a client's Google trends work across markets.

- How could do it in a way that reduced the manual work?

- How can we leverage Python's features and 3rd party libraries to automate this work?

- Does it make sense to automate it all the way?

### Part 1: Working with CSV files


#### 1 - Download the data

- Navigate to https://trends.google.com/trends/explore?date=2021-05-01%202021-05-31&geo=AU-ACT&q=%2Fm%2F08h9qn

- Download the csv file of the trends you want to chart

#### 2 - Import Pandas package

In [ ]:
import pandas as pd

#### 3 - Read in the data into a pandas DataFrame

In [ ]:
df = pd.read_csv('data/1_trends/officeworks/5-21_aus.csv', header=1, index_col="Day") 

##### Note:
- *We are using a "read_csv" method. If your file was excel you'd need to use "read_excel"*

- *We are using a "header" argument. This is tell pandas which row has the headers, notice it is "1", not "2" - why?"*

- *We are setting an "index column" we'll discuss this more when we look at Pandas next week*

#### 4 - Review the data

In [ ]:
df.head() # Shows the first 5 rows

In [ ]:
df.plot(figsize=(12,4)) # Plots the data

In [ ]:
df.mean() # Find the average metric

In [ ]:
df[df["Officeworks: (Australia)"]==100] # Find the day of the max metric

#### 5 - Read in market data for NSW

In [ ]:
df_nsw = pd.read_csv('data/1_trends/officeworks/5-21_nsw.csv', header=1, index_col="Day") 

In [ ]:
df_nsw.plot(figsize=(12,4))

#### 6 - Get all market data

In [ ]:
df_vic = pd.read_csv('data/1_trends/officeworks/5-21_vic.csv', header=1, index_col="Day") 
df_qld = pd.read_csv('data/1_trends/officeworks/5-21_qld.csv', header=1, index_col="Day") 

df_comb1 = df_nsw.join(df_vic)
df_combined = df_comb1.join(df_qld)

In [ ]:
df_combined.plot(figsize=(12,4))

#### 7 - How could we do that better?

In [ ]:
# 1) make a list of all the markets
markets = ["nsw", "vic", "qld"] 

# 2) make an empty list for the dataframes
dfs = []

# 3) loop through all the markets and append its dataframe to the list
for market in markets:
    csv_path = f"data/1_trends/officeworks/5-21_{market}.csv"
    dfs.append(pd.read_csv(csv_path, header=1, index_col="Day"))

# 4) combine the list of dataframes
df_combined = pd.concat(dfs, axis=1)


df_combined.plot(figsize=(12,4))

#### 8 - Exercise - Create the same chart for "Johnson & Johnson"?

#### 9 - Exercise - Create the same chart for "Lotterywest"?

#### 10 - How could our script be more flexible?


______________

### Part 2: Google Trends API

#### 1 - Install the pytrends library

In [ ]:
pip install pytrends

#### 2 - Follow the instructions to construct a request

In [ ]:
from pytrends.request import TrendReq
pytrends = TrendReq(hl='en-AU', tz=600) # create a new pytrends object

In [ ]:
kw_list = ["Officeworks"] # build a keyword list
pytrends.build_payload(kw_list,timeframe="2021-05-01 2021-05-31", geo="AU-VIC") #build a payload
df = pytrends.interest_over_time() # get the data
df.head()

#### 3 - Format the DataFrame to use original column names

In [ ]:
df.index.name="Day"
df.rename(columns={"Officeworks":"Officeworks: (Victoria)"}, inplace=True)
df.drop("isPartial", axis=1, inplace=True)
df.head()

#### 4 - Exercise build the plots again using pytrends package

In [ ]:
# First define constants

client_list = ["Officeworks", "Johnson & Johnson", "Lotterywest"]
market_dict = {
    "AU-NSW": "New South Wales",
    "AU-VIC": "Victoria",
    "AU-QLD": "Queensland",
}
FIG_SIZE = (12, 4)

In [ ]:
# Second define utility functions
def format_df(df, keyword, market):
    df.index.name="Day"
    market_name = market_dict[market]
    new_col_name = f"{keyword}: ({market_name})"
    df.rename(columns={keyword: new_col_name}, inplace=True)
    df.drop("isPartial", axis=1, inplace=True)
    return df

def get_trends_df(client, market, start_date, end_date):
    kw_list = [client] 
    pytrends.build_payload(kw_list,timeframe=f"{start_date} {end_date}", geo=market)
    df = pytrends.interest_over_time()
    return format_df(df, client, market)
    
    

In [ ]:
# Third define final script
def plot_trends(client_list, market_list, start_date, end_date):
    for client in client_list:
        dfs = []
        for market in market_list:
            print(f"Collecting trends for {client} in {market}")
            dfs.append(get_trends_df(client, market, start_date, end_date))
        pd.concat(dfs, axis=1).plot(figsize=FIG_SIZE)
    

    
    

In [ ]:
# Run script
plot_trends(client_list, market_dict,"2021-05-01", "2021-05-31" )